<!--Copyright © ZOMI 适用于[License](https://github.com/Infrasys-AI/AIInfra)版权许可-->

# CODE 01: 低精度推理性能对比

在大模型部署和应用中，推理效率和精度之间的平衡一直是一个关键挑战。随着模型规模的不断增长（从几亿参数到千亿参数），存储需求和计算开销也随之急剧增加。低精度推理技术通过使用更少的比特数来表示模型参数和计算中间结果，为解决这一问题提供了有效途径。

本文将以 Qwen3 4B 模型为研究对象，对比 FP16（作为基线）、FP8、FP6 和 INT8 四种精度在推理效率和精度上的表现，帮助读者理解不同精度量化对模型性能的影响。

## 1. 技术原理：量化基础

模型量化的核心思想是将神经网络中的浮点数参数和激活值从高精度（如 FP32）转换为低精度（如 FP16、INT8 等）表示。这一过程可以用以下公式表示：

对于整数量化，我们有：

$$ x_{int} = \text{round}(x_{float} / s + z) $$

其中，$s$ 是缩放因子（scale），$z$ 是零点（zero point），用于将浮点数映射到整数域。

对于浮点数量化（如 FP8），则是通过减少指数位和尾数位的数量来实现，这会直接影响数值的表示范围和精度。

量化带来的好处主要有三点：

1. 减少内存占用：例如 INT8 仅需 FP32 1/4 的存储空间
2. 提高计算效率：低精度计算通常更快，尤其在支持 SIMD 指令的硬件上
3. 降低功耗：低精度计算需要更少的能量

但量化也可能导致精度损失，这也是我们本次实验需要验证的重点。

## 2. 实验环境准备

首先，让我们准备实验所需的环境和库。我们将使用 Hugging Face 的 Transformers 库加载 Qwen3 4B 模型，使用 Accelerate 库进行分布式加速，并使用 Evaluate 库评估模型性能。

In [1]:
# 安装必要的库
!pip install transformers accelerate evaluate datasets bitsandbytes

接下来，导入所需的库：

In [1]:
import torch
import time
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from evaluate import load
from datasets import load_dataset
import matplotlib.pyplot as plt
import re

# 设置随机种子，保证实验可复现
torch.manual_seed(42)
np.random.seed(42)

/home/yswang/miniforge3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 3. 模型与数据集准备

我们将使用 Qwen3 4B 模型和一个常用的评估数据集。为了简化实验，我们选择了相对较小的数据集。

In [2]:
# 模型名称
#model_name = "Qwen/Qwen3-4B-Chat"
model_name = "Qwen/Qwen3-4B-Instruct-2507"  # 或者 "Qwen/Qwen3-4B" / "Qwen/Qwen3-4B-Thinking-2507"

# 加载分词器
tokenizer = AutoTokenizer.from_pretrained(model_name)
# 设置 padding 和截断策略
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

对于评估数据集，我们选择了常用的"lambada"数据集，它主要用于评估模型的句子续写能力：

In [3]:
# 加载评估数据集
# dataset = load_dataset("lambada")
dataset = load_dataset("EleutherAI/lambada_openai")

# 取前 100 个样本作为测试集（简化实验）
test_dataset = dataset["test"].select(range(100))

由于数据集在外网，因此可能出现`Connect Error`，如果你有clash，可以先看一下梯子的端口然后配置一下全局代理：
```bash
#比如我clash的端口是7897
git config --global http.proxy https://127.0.0.1:7899
git config --global https.proxy https://127.0.0.1:7899
git config --global lfs.https://huggingface.co/.proxy https://127.0.0.1:7899
```

如果网络确实不行，可以先下载到本地`AIInfra/05Infer/06CompDistill/data`：
```bash
pip install -U huggingface_hub

hf download EleutherAI/lambada_standard --repo-type dataset --local-dir ./data/lambada_standard
```

再从本地加载数据集:

In [ ]:
from datasets import load_dataset

dataset = load_dataset("./data/lambada_standard", split="test")  # 本地路径

test_dataset = dataset.select(range(100))

FileNotFoundError: Couldn't find any data file at /home/yswang/tvm_learn/cuda/AIInfra/05Infer/06CompDistill/data/lambada_standard.

让我们看看数据集中的样本是什么样子的：

In [4]:
# 查看一个样本
print("样本示例：")
print(test_dataset[0]["text"])

样本示例：
In my palm is a clear stone, and inside it is a small ivory statuette. A guardian angel.

"Figured if you're going to be out at night getting hit by cars, you might as well have some backup."

I look at him, feeling stunned. Like this is some sort of sign. But as I stare at Harlin, his mouth curved in a confident grin, I don't care about signs


## 4. 评估函数定义

在开始实验前，我们需要定义一个评估函数，用于计算模型在不同精度下的推理时间和准确率。

In [5]:
def evaluate_model(model, tokenizer, dataset, max_new_tokens=10): 
    model.eval()

    gen = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        return_full_text=False,            # 关键：只返回续写
    )

    def normalize_word(w: str) -> str:
        # 去掉词尾标点（保留字母/数字/下划线/撇号/连字符），再小写
        return re.sub(r"[^\w'-]+$", "", w).lower()

    total_time = 0.0
    correct = 0
    total = 0

    for item in dataset:
        text = item["text"]
        words = text.split()
        if len(words) < 2:
            continue  # 跳过异常样本

        prefix = " ".join(words[:-1])
        target_word = normalize_word(words[-1])

        # 计时：GPU 前后同步，使用高精度时钟
        if torch.cuda.is_available():
            torch.cuda.synchronize()
        t0 = time.perf_counter()

        with torch.inference_mode():       # 推理模式，禁用 autograd 以提速
            out = gen(
                prefix,
                max_new_tokens=max_new_tokens,   # 通常 1 即可
                do_sample=False,
                pad_token_id=tokenizer.eos_token_id,
            )[0]["generated_text"]

        if torch.cuda.is_available():
            torch.cuda.synchronize()
        total_time += (time.perf_counter() - t0)

        # 提取首个生成“词”
        # 允许包含连字符/撇号（e.g., we're, mother-in-law）
        m = re.match(r"\s*([A-Za-z]+(?:['-][A-Za-z]+)?)", out)
        pred_first = normalize_word(m.group(1)) if m else ""

        if pred_first == target_word:
            correct += 1
        total += 1

        if total % 10 == 0:
            print(f"完成 {total}/{len(dataset)} 个样本")

    accuracy = (correct / total) if total else 0.0
    avg_time_per_sample = (total_time / total) if total else 0.0
    return total_time, avg_time_per_sample, accuracy

## 5. FP16 精度实验

首先，我们以 FP16 精度作为基线进行实验。FP16（半精度浮点数）使用 16 位表示一个浮点数，相比 FP32（单精度）能节省一半的存储空间。

In [8]:
# 加载 FP16 精度的模型
# 可以先在命令行 export HF_HUB_DISABLE_XET=1 有效预防504超时
model_fp16 = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,  # 指定为 FP16 精度
    device_map="auto"  # 自动分配设备
)

# 评估 FP16 模型
print("开始评估 FP16 模型...")
total_time_fp16, avg_time_fp16, acc_fp16 = evaluate_model(
    model_fp16, 
    tokenizer, 
    test_dataset,
    max_new_tokens=1   
)

# 打印结果
print(f"FP16 - 总推理时间: {total_time_fp16:.2f}秒, "
      f"平均每个样本: {avg_time_fp16:.4f}秒, "
      f"准确率: {acc_fp16:.4f}")

`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 3/3 [00:02<00:00,  1.04it/s]
Device set to use cuda:0


开始评估 FP16 模型...
完成 10/100 个样本
完成 20/100 个样本
完成 30/100 个样本
完成 40/100 个样本
完成 50/100 个样本
完成 60/100 个样本
完成 70/100 个样本
完成 80/100 个样本
完成 90/100 个样本
完成 100/100 个样本
FP16 - 总推理时间: 4.57秒, 平均每个样本: 0.0457秒, 准确率: 0.5100


FP16 之所以被广泛用作基线，是因为它在精度损失相对较小的情况下，能显著提升推理速度并减少内存占用。对于大多数模型，从 FP32 转为 FP16 不会导致明显的精度下降，但能带来约 2 倍的性能提升。

## 6. INT8 精度实验

接下来，我们尝试 INT8 精度。INT8 使用 8 位整数表示数据，相比 FP16 能再减少一半的存储空间，即仅为 FP32 的 1/4。

In [9]:
# 加载 INT8 精度的模型
model_int8 = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,  # 启用 INT8 量化
    device_map="auto"
)

# 评估 INT8 模型
print("开始评估 INT8 模型...")
total_time_int8, avg_time_int8, acc_int8 = evaluate_model(
    model_int8, 
    tokenizer, 
    test_dataset,
    max_new_tokens=1   
)

# 打印结果
print(f"INT8 - 总推理时间: {total_time_int8:.2f}秒, "
      f"平均每个样本: {avg_time_int8:.4f}秒, "
      f"准确率: {acc_int8:.4f}")

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Loading checkpoint shards: 100%|██████████| 3/3 [00:07<00:00,  2.47s/it]
Device set to use cuda:7


开始评估 INT8 模型...
完成 10/100 个样本
完成 20/100 个样本
完成 30/100 个样本
完成 40/100 个样本
完成 50/100 个样本
完成 60/100 个样本
完成 70/100 个样本
完成 80/100 个样本
完成 90/100 个样本
完成 100/100 个样本
INT8 - 总推理时间: 12.12秒, 平均每个样本: 0.1212秒, 准确率: 0.5100


INT8 量化是目前应用最广泛的低精度技术之一，因为它在精度和性能之间取得了很好的平衡。其核心原理是将浮点范围映射到整数范围，通常使用最小-最大量化方法：

$$ x_{int8} = \text{clip}(\text{round}(x_{float} / s + 127), 0, 255) $$

其中 $s$ 是缩放因子，计算方式为：$s = \frac{\text{max}(|x_{float}|)}{127}$

## 7. FP8 精度实验

FP8 是一种较新的低精度浮点格式，相比 FP16 进一步减少了位数，但保留了浮点数的动态范围优势。

In [11]:
# 使用 bitsandbytes 库实现 FP8 量化
from bitsandbytes import quantization

# 首先加载 FP16 模型
model_fp8 = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

# 应用 FP8 量化
quantization.quantize_model(model_fp8, bits=8, quant_type="fp8")

# 评估 FP8 模型
print("开始评估 FP8 模型...")
total_time_fp8, avg_time_fp8, acc_fp8 = evaluate_model(
    model_fp8, 
    tokenizer, 
    test_dataset
)

# 打印结果
print(f"FP8 - 总推理时间: {total_time_fp8:.2f}秒, "
      f"平均每个样本: {avg_time_fp8:.4f}秒, "
      f"准确率: {acc_fp8:.4f}")

ImportError: cannot import name 'quantization' from 'bitsandbytes' (/home/yswang/miniforge3/lib/python3.12/site-packages/bitsandbytes/__init__.py)

FP8 有两种主要格式：E4M3（4 位指数，3 位尾数）和 E5M2（5 位指数，2 位尾数）。E4M3 提供更高的精度但范围较小，而 E5M2 则相反。在实际应用中，会根据具体场景选择合适的格式。

相比 INT8，FP8 在表示非常大和非常小的数值时更有优势，这使得它在某些场景下能保持比 INT8 更高的精度。

## 8. FP6 精度实验

FP6 是一种更激进的低精度格式，使用 6 位表示浮点数。由于位数更少，它的精度会受到更大影响，但理论上能提供更高的性能。

In [ ]:
# 加载基础模型
model_fp6 = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

# 实现 FP6 量化（简化版）
def quantize_to_fp6(tensor):
    """将张量量化为 FP6 精度（简化实现）"""
    # 在实际应用中，这会更复杂，需要考虑指数和尾数位的分配
    # 这里使用一种简单的缩放方法作为示例
    min_val = tensor.min()
    max_val = tensor.max()
    scale = (max_val - min_val) / 63  # 6 位可以表示 64 个值
    return ((tensor - min_val) / scale).round().clamp(0, 63)

# 对模型参数应用 FP6 量化
for param in model_fp6.parameters():
    param.data = quantize_to_fp6(param.data).float() / 63 * (param.data.max() - param.data.min()) + param.data.min()

# 评估 FP6 模型
print("开始评估 FP6 模型...")
total_time_fp6, avg_time_fp6, acc_fp6 = evaluate_model(
    model_fp6, 
    tokenizer, 
    test_dataset
)

# 打印结果
print(f"FP6 - 总推理时间: {total_time_fp6:.2f}秒, "
      f"平均每个样本: {avg_time_fp6:.4f}秒, "
      f"准确率: {acc_fp6:.4f}")

注意：上面的 FP6 量化是一个简化实现。在实际应用中，FP6 的实现会更复杂，通常采用 E2M3（2 位指数，3 位尾数）的格式。由于 FP6 的表示能力有限，它通常只用于对精度要求不高的场景，或者作为研究探索。

## 9. 实验结果对比

现在我们已经完成了所有精度的实验，让我们将结果汇总并进行分析。

In [ ]:
# 汇总结果
results = {
    "Precision": ["FP16", "FP8", "FP6", "INT8"],
    "Total Time (s)": [total_time_fp16, total_time_fp8, total_time_fp6, total_time_int8],
    "Avg Time per Sample (s)": [avg_time_fp16, avg_time_fp8, avg_time_fp6, avg_time_int8],
    "Accuracy": [acc_fp16, acc_fp8, acc_fp6, acc_int8],
    "Speedup vs FP16": [1.0, total_time_fp16/total_time_fp8, 
                       total_time_fp16/total_time_fp6, 
                       total_time_fp16/total_time_int8],
    "Accuracy Drop": [0.0, acc_fp16 - acc_fp8, 
                     acc_fp16 - acc_fp6, 
                     acc_fp16 - acc_int8]
}

# 打印结果表格
import pandas as pd
results_df = pd.DataFrame(results)
print(results_df)

让我们可视化这些结果，以便更直观地比较不同精度的表现：

In [ ]:
# 设置中文字体
plt.rcParams["font.family"] = ["SimHei", "WenQuanYi Micro Hei", "Heiti TC"]

# 创建对比图表
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# 速度对比
ax1.bar(results["Precision"], results["Speedup vs FP16"], color=['blue', 'green', 'red', 'purple'])
ax1.set_title('不同精度相对 FP16 的速度提升倍数')
ax1.set_ylabel('速度提升倍数')
ax1.grid(axis='y', linestyle='--', alpha=0.7)

# 精度对比
ax2.bar(results["Precision"], results["Accuracy"], color=['blue', 'green', 'red', 'purple'])
ax2.set_title('不同精度下的模型准确率')
ax2.set_ylabel('准确率')
ax2.set_ylim(0, 1.0)  # 准确率范围在 0-1 之间
ax2.grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()

从实验结果中，我们可以观察到以下几点：

1. 性能提升：一般来说，精度越低，推理速度越快。INT8 和 FP8 通常能提供 2-3 倍的速度提升，而 FP6 可能更快。

2. 精度损失：随着精度降低，模型准确率通常会有所下降。FP8 的精度损失通常较小，而 FP6 可能会有较明显的精度损失。

3. 权衡选择：INT8 通常在速度和精度之间提供最佳平衡，是实际应用中的首选；FP8 在需要更高精度的场景下表现更好；而 FP6 则适用于对速度要求极高但可以接受较大精度损失的场景。

## 总结与思考

本实验对比了不同低精度格式（FP16、FP8、FP6 和 INT8）对 Qwen3 4B 模型推理性能和精度的影响。实验结果表明，低精度推理确实能显著提升模型的推理速度，但也可能带来一定的精度损失。

这些发现对于大模型的实际部署具有重要指导意义，帮助开发者在不同的硬件条件和精度要求下选择合适的量化策略。